In [33]:
import pandas as pd
import numpy as np

import jieba
import jieba.posseg as pseg
import jieba.analyse

import sys
from optparse import OptionParser
import pprint
from prettytable import PrettyTable

In [34]:
df = pd.read_csv(open('../data/df_small.csv'))
df = df[df['text'].str.contains("https|Britain")== False]
print('有{}筆對話'.format(len(df)))

有91340筆對話


## groupby

In [35]:
grouped_data = df.groupby('from')['text'].apply(lambda x: '/'.join(x)).reset_index()
l = len(grouped_data)
print('有{}個使用者'.format(l))

有1019個使用者


## 先用前1000筆資料做測試

In [36]:
test = grouped_data.head(1000)
test_len = len(test)
test.head()

,from,text
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...


In [37]:
for i in range(len(test)):
    test.iloc[i][1] = test.iloc[i][1].split('/')
test.head()

,from,text
0,1000970273334199,"[找不到阿, 安安, 你有打LOL嗎, 好吧，可以帶我嗎, 安安, 凸, Sor, 嗯, 約..."
1,1001070263323689,"[!?, 唐寶寶, 你那麼愛慕他ㄇ, 嗨, 先問問您期望本人的性別是?, 了解, 我是男的,..."
2,1001967239901904,"[早, 在床上發懶, 爆, 斷, 魂, 你在暗諷綠藻嘛, 壞, 話說我好餓ㄛ, 窩不要這個q..."
3,1003172849781848,"[甲, 你要做我的0？, 人生勝利組是不會來玩這個的, 你一定不是c銘, 也不是業王, 葉你..."
4,1003929993039131,"[吃噓, 肥宅早, 我們不是場外人, 場外ㄈㄓ, 我沒用了, 安安, 是喔, 場外ㄈㄓ嗎, ..."


In [38]:
print('id為 {} 的使用者傳送的訊息{}'.format(test.iloc[0][0],test.iloc[0][1]))

id為 1000970273334199 的使用者傳送的訊息['找不到阿', '安安', '你有打LOL嗎', '好吧，可以帶我嗎', '安安', '凸', 'Sor', '嗯', '約哪', '有啊', '你先約的啊', '喔', '輸入0088', '我懶的打了', '皮米屌', '這你嗎？', '魏癢癢', '哪裡人?', '住哪裡啦？', '超癢', '哪裡人?', '好遠', '其實還好', '新竹', '龍潭?', 'ㄏㄅㄨㄐㄅㄌㄗㄋ', '三小', '幹又ㄋ', '掰掰', '凹?', '凸sor']


In [39]:
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
stopwordset

{'10',
 '20',
 '2016',
 '?',
 '他',
 '他們',
 '你',
 '台灣',
 '她',
 '對於',
 '成為',
 '我',
 '我們',
 '沒有',
 '王隆甫',
 '？'}

In [40]:
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?!$,:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

In [41]:
# stopwordset

## gensim model

In [42]:
from gensim import models
import gensim

In [43]:
sentences = models.word2vec.LineSentence("corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)
# model.wv.vocab

## pick user top 10 word ( frequency-based )

In [44]:
top10_vector = np.zeros(shape=(l,10))
# top10_vector

In [45]:
import operator
for i in range(l):
    og_dict = dict()
#     print('\n user'+str(i+1))
#     if (len(test.iloc[i][1]))<10:
#         continue
    for j in range(len(grouped_data.iloc[i][1])):
        seglist = jieba.cut((grouped_data.iloc[i][1])[j])
        for word in seglist:
            if word not in stopwordset:
                if not word in og_dict.keys():
                    if word == " ":
                        continue
                    og_dict[word] = 1
                else:
                    og_dict[word] = int(og_dict[word]) + 1
    a = sorted(og_dict.items(), key=operator.itemgetter(1),reverse=True)
#     print(a)
    
    row = np.zeros(shape=(1,10))
    
    if len(a) >= 10:
        for x in range(10):
            if a[x][0] in model:
                show = float(model[a[x][0]])
#                 print(show)
                row[0][x] = show
    else:
        for x in range(len(a)):
#             print(a[x]) 
            if a[x][0] in model:
                show = float(model[a[x][0]])
#                 print(show)
                row[0][x] = show
#     print(row)
    top10_vector[i] = row

Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
Loading model cost 1.514 seconds.
Prefix dict has been built succesfully.


In [46]:
top10_vector.shape

(1019, 10)

In [ ]:
# top10_vector

## 分群

In [52]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3,random_state=1170).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果 前10：")
print(cluster_labels[:10])
print("---")




分群結果 前10：
[1 1 1 2 1 1 1 2 1 0]
---


## 分群結果數量

In [53]:
unique, counts = np.unique(cluster_labels, return_counts=True)
dict(zip(unique, counts))


{0: 109, 1: 557, 2: 353}

In [54]:
grouped_data['category'] = cluster_labels

In [55]:
grouped_data.head(20)

,from,text,category
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...,1
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...,1
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...,1
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...,2
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...,1
5,1006942526092443,安/是/靠北哈哈/是要說啥/呃/對呀/好/那我們來玩個遊戲/1188-1100/扣掉的要補零...,1
6,1007352782715651,嗨/......./看來是沒有/我是有差點想打/哈哈/真的離開喔/想說多打一句好了/哈哈/下...,1
7,1018989321532115,Hi/O.O/Ooo/黃色那隻/已知啊/Hi/還沒睡啊/不睡啊,2
8,1020089304775462,嗨/嗨/斷/嗨/哪裡人/挖/國外耶/天龍國人啊/感覺不同世界/嗨/安安 幾歲 住那 浪...,1
9,1020229148084642,安安,0
